In [1]:
import ee
import geemap
#port根据自己科学工具的端口号修改
geemap.set_proxy(port=4780)
Map = geemap.Map(center=(30, 120), zoom=4)
table = ee.FeatureCollection("users/yamiletsharon250/wuhan")
roi = table
Map.addLayer(roi, {}, "武汉")
Map

Map(center=[30, 120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [2]:
#获取影像与去云

#l457 去云函数
def rmL457Cloud(image):
    qa = image.select('pixel_qa')
    cloud = qa.bitwiseAnd(1 << 5) \
                    .And(qa.bitwiseAnd(1 << 7)) \
                    .Or(qa.bitwiseAnd(1 << 3))
    mask2 = image.mask().reduce(ee.Reducer.min())
    mask3 = image.select('B1').gt(2000)
    return image.updateMask(cloud.Not()).updateMask(mask2).updateMask(mask3.Not()) \
                .copyProperties(image) \
                .copyProperties(image, ["system:time_start",'system:time_end','system:footprint'])

#l8 去云函数
def rmL8Cloud(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    mask2 = image.select('B1').gt(2000)
    return image.updateMask(mask).updateMask(mask2.Not()) \
                .copyProperties(image) \
                .copyProperties(image, ["system:time_start",'system:time_end'])
                
#定义影像执行云量筛选与去云
l8_sr = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").map(rmL8Cloud) \
              .filter(ee.Filter.lte('CLOUD_COVER',10))

l7_sr = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").map(rmL457Cloud) \
              .filter(ee.Filter.lte('CLOUD_COVER',10))

l5_sr = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR").map(rmL457Cloud) \
              .filter(ee.Filter.lte('CLOUD_COVER',10))

In [3]:
#计算水体方法_ARWDR，可以查阅论文，我忘了是哪篇了。。。
#获取dem山体角,意义在于去除山体阴影
def calcWater(image):
    MNDWI = image.select("MNDWI")
    NDWI = image.select("MNDWI")
    NDVI = image.select("NDVI")
    EVI = image.select("EVI")
    water = NDWI.gt(0) \
                   .And(MNDWI.gt(0.1) \
                   .And(EVI.lt(0.1)) \
                   .And(MNDWI.gt(NDVI)) \
                   .Or(MNDWI.gt(EVI)))
    dem = ee.Image("USGS/GMTED2010").select(0)
    azimuth = ee.Number(image.get('SUN_AZIMUTH'))
    ele = ee.Number(image.get('SUN_ELEVATION'))
    shadow = ee.Algorithms.HillShadow(dem, azimuth, ele).eq(1)
    return image.addBands(water.rename("water")).updateMask(shadow)

In [4]:
#Landsat-5/7处理方法

    #Landsat SR数据需要缩放，比例是 0.0001#
def Landsat57scaleImage(image):
      time_start = image.get("system:time_start")
      SUN_AZIMUTH = image.get('SOLAR_AZIMUTH_ANGLE')
      SUN_ELEVATION = image.get('SOLAR_ZENITH_ANGLE')
      image = image.select(["B1","B2","B3","B4","B5","B7"])
      image = image.divide(10000)
      image = image.set("system:time_start", time_start) \
                  .set("SUN_AZIMUTH",SUN_AZIMUTH ) \
                   .set("SUN_ELEVATION", SUN_ELEVATION)
      return image

    # SR数据去云#
def Landsat57srCloudMask(image):
      qa = image.select('pixel_qa')
      cloudShadowBitMask = (1 << 3)
      snowBitMask = (1 << 4)
      cloudsBitMask = (1 << 5)
      mask1 = qa.bitwiseAnd(cloudsBitMask).eq(0) \
                    .And(qa.bitwiseAnd(snowBitMask).eq(0)) \
                    .And(qa.bitwiseAnd(cloudShadowBitMask).eq(0))
      mask2 = image.mask().reduce(ee.Reducer.min())
      return image.updateMask(mask1.And(mask2))


    #NDVI
def Landsat57NDVI(image):
        return image.addBands(image.normalizedDifference(["B4", "B3"]) \
                                   .rename("NDVI"))
    

def Landsat57NDWI(image):
        return image.addBands(image.normalizedDifference(["B2", "B4"]) \
                                   .rename("NDWI"))
    #MNDWI

def Landsat57MNDWI(image):
        return image.addBands(image.normalizedDifference(["B2", "B5"]) \
                                   .rename("MNDWI"))

    # EVI
def Landsat57EVI(image):
      evi = image.expression("EVI = 2.5 * (NIR - R) / (NIR + 6*R -7.5*B + 1)", {
        'NIR': image.select("B4"),
        'R': image.select("B3"),
        'B': image.select("B1")
      })
      return image.addBands(evi)


    #获取Landsat5的SR数据，并select water#
def Landsat57getL5SRCollection(startDate, endDate, region):
      dataset = l5_sr.filterDate(startDate, endDate) \
                        .filterBounds(region) \
                        .map(Landsat57srCloudMask) \
                        .map(Landsat57scaleImage) \
                        .map(Landsat57NDVI) \
                        .map(Landsat57MNDWI) \
                        .map(Landsat57NDWI) \
                        .map(Landsat57EVI) \
                        .map(calcWater) \
                        .select("water")
      return dataset

    #* 获取Landsat7的SR数据#
def Landsat57getL7SRCollection(startDate, endDate, region):
      dataset = l7_sr.filterDate(startDate, endDate) \
                        .filterBounds(region) \
                        .map(Landsat57srCloudMask) \
                        .map(Landsat57scaleImage) \
                        .map(Landsat57NDVI) \
                        .map(Landsat57NDWI) \
                        .map(Landsat57MNDWI) \
                        .map(Landsat57EVI) \
                        .map(calcWater) \
                        .select("water")
      return dataset

In [5]:
#Landsat 8处理方法

    # Landsat8 SR数据缩放#
def Landsat8scaleImage(image):
      time_start = image.get("system:time_start")
      SUN_AZIMUTH = image.get('SOLAR_AZIMUTH_ANGLE')
      SUN_ELEVATION = image.get('SOLAR_ZENITH_ANGLE')
      image = image.select(["B2","B3","B4","B5","B6","B7"])
      image = image.divide(10000)
      image = image.set("system:time_start", time_start) \
                   .set("SUN_AZIMUTH",SUN_AZIMUTH ) \
                   .set("SUN_ELEVATION", SUN_ELEVATION)
      return image


    #SR数据去云#
def Landsat8srCloudMask(image):
      cloudShadowBitMask = (1 << 3)
      snowBitMask = (1 << 4)
      cloudsBitMask = (1 << 5)
      qa = image.select('pixel_qa')
      mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
      return image.updateMask(mask)


    #NDVI
def Landsat8NDVI(image):
        return image.addBands(image.normalizedDifference(["B5", "B4"]) \
                                   .rename("NDVI"))
def Landsat8NDWI(image):
        return image.addBands(image.normalizedDifference(["B3", "B5"]) \
                                   .rename("NDWI"))

    #MNDWI
def Landsat8MNDWI(image):
        return image.addBands(image.normalizedDifference(["B3", "B6"]) \
                                   .rename("MNDWI"))


    # EVI
def Landsat8EVI(image):
      evi = image.expression("EVI = 2.5 * (NIR - R) / (NIR + 6*R -7.5*B + 1)", {
        'NIR': image.select("B5"),
        'R': image.select("B4"),
        'B': image.select("B2")
      })
      return image.addBands(evi)


    #获取Landsat8的SR数据#
def Landsat8getL8SRCollection(startDate, endDate, region):
      dataset = l8_sr.filterDate(startDate, endDate) \
                        .filterBounds(roi) \
                        .map(Landsat8srCloudMask) \
                        .map(Landsat8scaleImage) \
                        .map(Landsat8NDVI) \
                        .map(Landsat8MNDWI) \
                        .map(Landsat8NDWI) \
                        .map(Landsat8EVI) \
                        .map(calcWater) \
                        .select("water")
      return dataset

In [6]:
#生成逐年的水体
def processYearWaterImage(year, region):
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate = ee.Date.fromYMD(year+1, 1, 1)
    l5Water = Landsat57getL5SRCollection(startDate, endDate, region)
    l7Water = Landsat57getL7SRCollection(startDate, endDate, region)
    l8Water = Landsat8getL8SRCollection(startDate, endDate, region)
    waterImgs = l5Water.merge(l7Water).merge(l8Water)
    #计算水体的频率#
    waterImg = waterImgs.sum() \
                            .divide(waterImgs.count()) \
                            .clip(region)
    #提高置信度，和前面山体阴影一样，不删是因为怕报错#
    hand = ee.ImageCollection('users/gena/global-hand/hand-100')
    hand30 = hand.mosaic().focal_mean(0.1).rename('elevation')
    hillShadowMask = hand30.select('elevation').lte(50)
    dem = ee.Image("USGS/GMTED2010").select(0)
    slope = ee.Terrain.slope(dem)
    waterImg = waterImg.updateMask(hillShadowMask).updateMask(slope.lt(20));#mask外 is NoData
    waterImg = waterImg.unmask(0).clip(region)
    waterImg = waterImg.where(waterImg.select([0]).gt(0).And(waterImg.select([0]).lte(0.25)),0) \
                      .where(waterImg.select([0]).gt(0.25).And(waterImg.select([0]).lte(0.75)),1) \
                      .where(waterImg.select([0]).gt(0.75).And(waterImg.select([0]).lte(1)),2) \
                      .toUint8() \
                      .clip(roi)
    #范围从34（年份：1985）到1（年份：2018)所以是year-1985
    butoushuiimg = ee.Image('Tsinghua/FROM-GLC/GAIA/v10').select("change_year_index").eq(year-1985).clip(roi)
    year = str(year)
    Map.addLayer(butoushuiimg, {'min':0, 'max':0, 'opacity':1,'palette':['black']},year+"不透水面", False)
    Map.addLayer(waterImg, {'min':0, 'max':2, 'opacity':1,'palette':['ffffff','red','blue']},year+"ARWDR", False)
    #交集取反，留waterimg
    difference = waterImg.updateMask(butoushuiimg.mask().Not()); 
    Map.addLayer(difference, {'min':0, 'max':2, 'opacity':1,'palette':['ffffff','red','blue']},year+"交集", False)
    return waterImg
  
#循环导出所有的水体
def main():
    #开始年份和结束年份
    imglist= []
    year = 2015
    endyear = 2015
    while (year <= endyear):
      print(year)
      processYearWaterImage(year, roi)
      imglist.append(processYearWaterImage(year, roi))
      year = year + 1
    #生成水体imagecol，方便导出
    imglist = ee.ImageCollection(imglist)
    #时间序列动画,有需要可以打开,不然有点卡
    # labels = imglist.aggregate_array("system:index").getInfo()
    # Map.add_time_slider(imglist, {'min':0, 'max':2, 'opacity':0.7,'palette':['ffffff','red','blue']}, labels=labels, time_interval=3)

    
main()
Map

2015


Map(center=[30, 120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…